In [1]:
import os # importing OS library
from email.parser import Parser
import matplotlib.pyplot as plt
import math
from glob import glob
import multiprocessing
import matplotlib.pyplot as plt
import itertools 
import operator # importing operator
from collections import Counter  # importing counter
import csv # importing csv

In [2]:
rootDir = os.path.dirname(os.path.realpath('__file__'))

In [3]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    email_list = []
    if line:
        line = line.replace("\n", "")
        line = line.replace("\t", "")
        line = line.replace(" ", "")

        line = line.split(",")

        for email_to_1 in line:
            email_list.append(email_to_1)
    else:
        email_list = None
    return email_list

# def split_email_addresses(line):
#     '''To separate multiple email addresses'''
#     if line:
#         addrs = line.split(',')
#         addrs = frozenset(map(lambda x: x.strip(), addrs))
#     else:
#         addrs = None
#     return addrs

In [4]:
def email_analyse(filename, listOfEmails):
    with open(filename, "r") as f:
        data = f.read()
        email = Parser().parsestr(data)
        email_header = {}
        email_header['file'] =  filename.split('/')[-2:]
        email_header['content'] = get_text_from_email(email)
        for key in email.keys():
            email_header[key] = email.get(key)
        if email.get('To'):
            email_header['To'] = split_email_addresses(email_header['To'])
        if email.get('From'):
            email_header['From'] = split_email_addresses(email_header['From'])
        if email.get('Bcc'):
            email_header['Bcc'] = split_email_addresses(email_header['Bcc'])
        if email.get('Cc'):
            email_header['Cc'] = split_email_addresses(email_header['Cc'])
        if email.get('X-To'):
            email_header['X-To'] = split_email_addresses(email_header['X-To'])
        email_header['user'] =  filename.split('/')[-2]
        listOfEmails.append(email_header)

In [5]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [6]:
def get_listof_useremails(listOfEmails, path):
    include = ['sent', 'sent_items']
    [[email_analyse(os.path.join(directory, filename), listOfEmails) for filename in filenames] for directory, subdirectory, filenames in  os.walk(path) if directory.split('/')[-1] in include]

In [7]:
users = get_immediate_subdirectories(rootDir+'/data/enron/maildir/')

In [8]:
listOfEmails = []
if __name__ == '__main__':
    jobs = []
    for user in users :
        path = rootDir+'/data/enron/maildir/'+user+'/'
        p = multiprocessing.Process(target=get_listof_useremails(listOfEmails, path))
        jobs.append(p)
        p.start()

In [9]:
listOf1To1Email = [(email.get('From')[0],email.get('To')[0]) for email in listOfEmails if email.get('To') and len(email.get('To')) == 1]

In [10]:
sortedDict = sorted(dict(Counter(listOf1To1Email)).items(), key=operator.itemgetter(1), reverse = 1) # sorting wordlist dictionary by value

In [11]:
with open(rootDir+'/ListOf1To1Emails.csv', 'a', encoding='utf-8') as outcsv:   
    #configure writer to write standard csv file
    writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    writer.writerow(['Sender', 'Receiver', 'Number of emails'])
    for (efrom, eto), no in sortedDict:
        #Write item to outcsv
        writer.writerow([efrom, eto, no])

In [12]:
print('\t'+'Sender'+'\t\t\t'+'Receiver'+'\t\t\t'+'Number of emails exchanged')
print('-'*90)
for (efrom, eto), no in sortedDict[:10]:
    print(efrom+'\t\t'+eto+'\t\t'+str(no))

	Sender			Receiver			Number of emails exchanged
------------------------------------------------------------------------------------------
vince.kaminski@enron.com		vkaminski@aol.com		1133
j.kaminski@enron.com		vkaminski@aol.com		440
kay.mann@enron.com		suzanne.adams@enron.com		439
vince.kaminski@enron.com		shirley.crenshaw@enron.com		311
matthew.lenhart@enron.com		erichardson@sarofim.com		282
kay.mann@enron.com		nmann@erac.com		275
kay.mann@enron.com		kathleen.carnahan@enron.com		237
robin.rodrigue@enron.com		gabriel.monroy@enron.com		229
kate.symes@enron.com		evelyn.metoyer@enron.com		229
jeffrey.shankman@enron.com		jennifer.burns@enron.com		227


In [13]:
listOf1To1OutOfEnronEmail = [(email.get('From')[0],email.get('To')[0]) for email in listOfEmails if email.get('To') and len(email.get('To')) == 1 and 'enron' not in (email.get('To')[0]).lower()]

In [14]:
sortedDict = sorted(dict(Counter(listOf1To1OutOfEnronEmail)).items(), key=operator.itemgetter(1), reverse = 1) # sorting wordlist dictionary by value

In [15]:
with open(rootDir+'/ListOf1To1OutOfEnronEmail.csv', 'a', encoding='utf-8') as outcsv:   
    #configure writer to write standard csv file
    writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    writer.writerow(['Sender', 'Receiver', 'Number of emails'])
    for (efrom, eto), no in sortedDict:
        #Write item to outcsv
        writer.writerow([efrom, eto, no])

In [16]:
print('\t'+'Sender'+'\t\t\t'+'Receiver'+'\t\t\t'+'Number of emails exchanged')
print('-'*90)
for (efrom, eto), no in sortedDict[:10]:
    print(efrom+'\t\t'+eto+'\t\t'+str(no))

	Sender			Receiver			Number of emails exchanged
------------------------------------------------------------------------------------------
vince.kaminski@enron.com		vkaminski@aol.com		1133
j.kaminski@enron.com		vkaminski@aol.com		440
matthew.lenhart@enron.com		erichardson@sarofim.com		282
kay.mann@enron.com		nmann@erac.com		275
matthew.lenhart@enron.com		mmmarcantel@equiva.com		209
jeff.dasovich@enron.com		nancy.sellers@robertmondavi.com		185
chris.germany@enron.com		ingrid.immer@williams.com		161
matthew.lenhart@enron.com		shirley.s.elliott@citicorp.com		160
matthew.lenhart@enron.com		shelliott@dttus.com		130
matthew.lenhart@enron.com		tlenhart@corealty.com		129
